# Facial Expression Recognition video stream

You can get the shape_predictor_68_face_landmarks.dat file from:
 [shape_predictor_68_face_landmarks.dat.bz2](http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2)

In [1]:
#Import required modules
import cv2, glob, random, math, numpy as np, dlib, itertools
from sklearn.externals import joblib

emotions = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sadness", "surprise"]

#Set up some required objects
#Webcam object
video_capture = cv2.VideoCapture(0)
ret, frame = video_capture.read()
# width = np.size(frame, 1)
# height = np.size(frame, 0)
#Face detector
detector = dlib.get_frontal_face_detector() 
#Landmark identifier. Set the filename to whatever you named the downloaded file
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# load facial expression model SVM
clf2 = joblib.load('SVC_model.pkl')

while True:
    ret, frame = video_capture.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(gray)

    #Detect the faces in the image
    detections = detector(clahe_image, 1)
    
    #For each detected face
    for k,d in enumerate(detections): 
        #Get coordinates
        shape = predictor(clahe_image, d) 
        #Draw Facial Landmarks with the predictor class
        xlist, ylist = [], []
        #Store X and Y coordinates in two lists
        for i in range(1,68):
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
            #For each point, draw a red circle with thickness2 on the original frame
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2)
        
        #Get the mean of both axes to determine centre of gravity
        xmean, ymean = np.mean(xlist), np.mean(ylist)
        #get distance between each point and the central point in both axes
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]

        #If x-coordinates of the set are the same, the angle is 0, 
        #catch to prevent 'divide by 0' error in function
        if xlist[26] == xlist[29]:
            anglenose = 0
        else:
            anglenose = int(math.atan((ylist[26]-ylist[29])/(xlist[26]-xlist[29]))*180/math.pi)

        if anglenose < 0:
            anglenose += 90
        else:
            anglenose -= 90

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x)
            landmarks_vectorised.append(y)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            anglerelative = (math.atan((z-ymean)/(w-xmean))*180/math.pi) - anglenose
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append(anglerelative)

        ind = clf2.predictor(landmarks_vectorised)  
        #show facial expression on the screen
        cv2.putText(frame, emotions[ind], (10, 10), cv2.FONT_HERSHEY_PLAIN, 1.0, (0,255,0), thickness=1)

    cv2.imshow("image", frame) #Display the frame

    if cv2.waitKey(1) & 0xFF == ord('q'): #Exit program when the user presses 'q'
        break

NameError: name 'image' is not defined

## Extracting features from the faces
to transform these nice dots overlaid on your face into features to feed the classifer.

In [1]:
import cv2, glob, random, math, numpy as np, dlib, itertools
from sklearn.svm import SVC

#Emotion list
emotions = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sadness", "surprise"]
# emotions = ["neutral", "anger", "disgust", "happy", "surprise"]

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
#Set the classifier as a support vector machines with polynomial kernel
clf = SVC(kernel='linear', probability=True, tol=1e-3)

#Define function to get file list, randomly shuffle it and split 80/20
def get_files(emotion):
    files = glob.glob("dataset/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

def get_landmarks(image):
    detections = detector(image, 1)
    #For all detected face instances individually
    for k,d in enumerate(detections):
        #Draw Facial Landmarks with the predictor class
        shape = predictor(image, d)
        xlist = []
        ylist = []
        #Store X and Y coordinates in two lists
        for i in range(1,68):
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
        
        #Get the mean of both axes to determine centre of gravity
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        #get distance between each point and the central point in both axes
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]

        #If x-coordinates of the set are the same, the angle is 0, 
        #catch to prevent 'divide by 0' error in function
        if xlist[26] == xlist[29]:
            anglenose = 0
        else:
            anglenose = int(math.atan((ylist[26]-ylist[29])/(xlist[26]-xlist[29]))*180/math.pi)

        if anglenose < 0:
            anglenose += 90
        else:
            anglenose -= 90

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x)
            landmarks_vectorised.append(y)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            anglerelative = (math.atan((z-ymean)/(w-xmean))*180/math.pi) - anglenose
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append(anglerelative)

    if len(detections) < 1: 
        landmarks_vectorised = "error"
    return landmarks_vectorised

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            #open image
            image = cv2.imread(item)
            #convert to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe_image = clahe.apply(gray)
            landmarks_vectorised = get_landmarks(clahe_image)
            if landmarks_vectorised == "error":
                pass
            else:
                #append image array to training data list
                training_data.append(landmarks_vectorised)
                training_labels.append(emotions.index(emotion))
    
        for item in prediction:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe_image = clahe.apply(gray)
            landmarks_vectorised = get_landmarks(clahe_image)
            if landmarks_vectorised == "error":
                pass
            else:
                prediction_data.append(landmarks_vectorised)
                prediction_labels.append(emotions.index(emotion))

    return training_data, training_labels, prediction_data, prediction_labels   


In [2]:
accur_lin = []
for i in range(0,10):
    #Make sets by random sampling 80/20%
    print("Making sets %s" %i)
    training_data, training_labels, prediction_data, prediction_labels = make_sets()

    npar_train = np.array(training_data) #Turn the training set into a numpy array for the classifier
    npar_trainlabs = np.array(training_labels)
    print("training SVM linear %s" %i) #train SVM
    clf.fit(npar_train, training_labels)

    print("getting accuracies %s" %i) #Use score() function to get accuracy
    npar_pred = np.array(prediction_data)
    pred_lin = clf.score(npar_pred, prediction_labels)
    print "linear: ", pred_lin
    accur_lin.append(pred_lin) #Store accuracy in a list

#Get mean accuracy of the 10 runs
print("Mean value lin svm: %.3f" %np.mean(accur_lin))

Making sets 0


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:61: RuntimeWarning: divide by zero encountered in double_scalars


training SVM linear 0
getting accuracies 0
linear:  0.964912280702
Making sets 1
training SVM linear 1
getting accuracies 1
linear:  0.877192982456
Making sets 2
training SVM linear 2
getting accuracies 2
linear:  0.877192982456
Making sets 3
training SVM linear 3
getting accuracies 3
linear:  0.921052631579
Making sets 4
training SVM linear 4
getting accuracies 4
linear:  0.90350877193
Making sets 5
training SVM linear 5
getting accuracies 5
linear:  0.90350877193
Making sets 6
training SVM linear 6
getting accuracies 6
linear:  0.885964912281
Making sets 7
training SVM linear 7
getting accuracies 7
linear:  0.894736842105
Making sets 8
training SVM linear 8
getting accuracies 8
linear:  0.938596491228
Making sets 9
training SVM linear 9
getting accuracies 9
linear:  0.929824561404
Mean value lin svm: 0.910


In [5]:
# create svm model
# emotion lists
emotions = ["anger", "contempt", "disgust", "fear", "happy", "neutral", "sadness", "surprise"]

# create a clahe from cv2
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# using delib detector and predictor to get face landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
#Set the classifier as a support vector machines with polynomial kernel
clf = SVC(kernel='linear', probability=True, tol=1e-3)

data, data_labels = [], []

for emotion in emotions:
    files = glob.glob("dataset/%s/*" % emotion)
    #Append data to training and prediction list, and generate labels 0-7
    for item in files:
        #open image
        image = cv2.imread(item)
        #convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe_image = clahe.apply(gray)
        landmarks_vectorised = get_landmarks(clahe_image)
        if landmarks_vectorised == "error":
            pass
        else:
            #append image array to training data list
            data.append(landmarks_vectorised)
            data_labels.append(emotions.index(emotion))

npar_train = np.array(data) #Turn the training set into a numpy array for the classifier
npar_trainlabs = np.array(data_labels)
clf.fit(npar_train, npar_trainlabs)

from sklearn.externals import joblib
joblib.dump(clf, 'SVC_model.pkl')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:61: RuntimeWarning: divide by zero encountered in double_scalars


['SVC_model.pkl']

In [ ]:
clf2 = joblib.load('SVC_model.pkl')
clf2.prdict()